<a href="https://colab.research.google.com/github/azizalwatban/College/blob/main/mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Earth Engine Python API
import ee
ee.Authenticate()
ee.Initialize(project ='ee-azooozalwatban')

In [ ]:
import geemap
import geemap.colormaps as cm


In [ ]:
# L8 cloud masking based on the QA pixel

def maskL8sr(image):
  # Get the pixel QA band.
  qa = image.select('QA_PIXEL')
  # Bits 3 and 4 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = int(2**4)
  cloudsBitMask = int(2**3)
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(\
          qa.bitwiseAnd(cloudsBitMask).eq(0))
  masked = image.updateMask(mask)
  return masked

In [ ]:
# get NDVI
def getNDVI_coll(image):
  ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
  addNdvi = image.addBands(ndvi)
  return addNdvi

In [ ]:
# get NDVI
def getNDWI_coll(image):
  ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
  addNdWi = image.addBands(ndwi)
  return addNdWi

In [ ]:
def getL8Image(startDate, endDate, roi, Cloud_limit):

  L8_collection = (
      ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
      .filterBounds(roi)
      .filterDate(startDate,endDate)
      .filter(ee.Filter.lt('CLOUD_COVER_LAND', Cloud_limit))
    )
  return L8_collection

In [ ]:
def getS2Image(startDate, endDate, roi):

  S2_collection = (
      ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterBounds(roi)
      .filterDate(startDate,endDate)
    )
  return S2_collection

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

## Graph Setting


In [ ]:
#Map
ndviParams = {'bands': ['NDVI'],
              'palette': ['red', 'orange', 'yellow', 'green'],
              'min': -1,
              'max': 1}

cloudParams = {'bands': ['B4', 'B3', 'B2'],
              'min': 0,
              'max': 30000}

## Collect Images

In [ ]:
#roi = ee.Geometry.Point([47.136946, 24.045290]).buffer(1000)
roi = ee.Geometry.Polygon(
            [[[
              42.883632607886085,
              17.13527079900794
            ],
            [
              42.883632607886085,
              16.958864699075207
            ],
            [
              43.08011771302151,
              16.958864699075207
            ],
            [
              43.08011771302151,
              17.13527079900794
            ],
            [
              42.883632607886085,
              17.13527079900794
            ]]])

roi = ee.Geometry.Polygon(
            [
                [
            [
              35.37454179628824,
              28.01266189119862
            ],
            [
              35.37454179628824,
              28.369785459938257
            ],
            [
              34.90817456283892,
              28.369785459938257
            ],
            [
              34.90817456283892,
              28.01266189119862
            ],
            [
              35.37454179628824,
              28.01266189119862
            ]
              ]
             ]
            )

export_roi = ee.Geometry.Polygon(
            [
                [
            [
              35.32991948094113,
              27.936471034035833
            ],
            [
              35.32991948094113,
              28.162380229726026
            ],
            [
              34.479714299157166,
              28.162380229726026
            ],
            [
              34.479714299157166,
              27.936471034035833
            ],
            [
              35.32991948094113,
              27.936471034035833
            ]
              ]
             ]
            )

Cloud_limit = 20

startDate = '2024-01-03'
endDate = '2024-01-07'

S2_images = (
      ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterBounds(roi)
      .filterDate(startDate, endDate)
    )

# get info
S2_image = S2_images.first().clip(roi)

## Mosaic

In [ ]:
# Spatially mosaic the images in the collection and display.
s2_mosaic = S2_images.mosaic()
m = geemap.Map()
m.center_object(roi)
m.addLayer(
    s2_mosaic,
    {'min': 0.0, 'max': 2000, 'bands': ['B4', 'B3', 'B2']},
    'RGB')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Mask Collection

In [84]:
S2_masked = S2_images.map(mask_s2_clouds)

#S2_masked = S2_masked.sort('CLOUD_COVER_LAND')

In [ ]:
Map2 = geemap.Map()
Map2.center_object(roi)
Map2.addLayer(S2_masked.first(),vis_params=cloudParams,name="mask")
Map2.addLayerControl()
Map2

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
Map3 = geemap.Map()
Map3.center_object(roi)
Map3.addLayer(
    S2_image,
    {'min': 0.0, 'max': 2000, 'bands': ['B4', 'B3', 'B2']},
    'RGB')
Map3

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Calculate NDWI

In [76]:
s2_ndwi = S2_images.map(getNDWI_coll)

NDWI = s2_ndwi.select('NDWI').first()
NDWI = s2_mosaic.normalizedDifference(['B3', 'B8']).rename('NDWI');

palette = ['red', 'yellow', 'green', 'cyan', 'blue']
palette = cm.palettes.ndwi
vis_params = {'min': 0, 'max': 4000, 'palette': palette}

Map4 = geemap.Map()
Map4.center_object(roi)
Map4.addLayer(
    NDWI,
    {'palette': palette},
    'NDWI')
Map4.add_colorbar(vis_params, label="Water", layer_name="NDWI")
Map4.addLayerControl()
Map4




Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [77]:
#Create NDWI mask
ndwiThreshold = NDWI.gte(0.0)
ndwiMask = ndwiThreshold.updateMask(ndwiThreshold)
Map5 = geemap.Map()
Map5.center_object(roi)
Map5.addLayer(
    ndwiThreshold,
    {'palette':['black', 'white']},
    'NDWI Binary Mask');
Map5.addLayer(ndwiMask, {'palette':['blue']}, 'NDWI Mask')
Map5

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Calculate the masked areas pixel area

In [ ]:
ndwiPixelArea = ee.Image.pixelArea().addBands(
            ndwiMask.select('NDWI')).divide(1e6).reduceRegion(
            reducer= ee.Reducer.sum().group(1),
            geometry= roi,
            scale= 10,
          )

list = ee.List(ee.Dictionary(ndwiPixelArea).get('groups'))
group0 = ee.Dictionary(list.get(0));

group0
display("Water area (km2):", group0.get('sum'))

'Water area (km2):'

## Modified Normalized Difference Water Index (MNDWI)

In [78]:
MNDWI = s2_mosaic.updateMask(ndwiMask).normalizedDifference(['B3', 'B11']).rename('MNDWI')


palette = ['red', 'yellow', 'green']
palette = cm.palettes.ndwi
vis_params = {'min': -0.4, 'max': 0.4, 'palette': palette}

Map5 = geemap.Map()
Map5.center_object(roi)
water_layer = Map5.addLayer(
    MNDWI,
    vis_params,
    #{'palette': palette},
    'MNDWI')
Map5.add_colorbar(vis_params, label="Water", layer_name="MNDWI")
Map5.addLayerControl()
Map5

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Export Image

In [ ]:
visualized_image = MNDWI.visualize(min=-0.4, max=0.4, palette=palette)

# Define output path for saving the GeoTIFF in your Google Drive
output_drive_path = '/content/drive/My Drive/MNDWI.tif'

# Use geemap to export the image directly to your Google Drive
geemap.ee_export_image(visualized_image, filename=output_drive_path, scale=20, region=export_roi, file_per_band=False)


Generating URL ...
Please wait ...
Data downloaded to /content/drive/My Drive/MNDWI.tif


## Normalized Difference Turbidity Index

In [79]:
startDate = '2018-05-03'
endDate = '2018-05-07'

S2_images = (
      ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterBounds(roi)
      .filterDate(startDate, endDate)
    )
s2_mosaic = S2_images.mosaic()


NDTI = s2_mosaic.updateMask(ndwiMask).normalizedDifference(['B11', 'B12']).rename('NDWI')


palette = ['red', 'yellow', 'green']
#palette = cm.palettes.BrBG_r
vis_params = {'min': -0.15, 'max': 0.25, 'palette': palette}

Map5 = geemap.Map()
Map5.center_object(roi)
Map5.addLayer(
    NDTI,
    vis_params,
    #{'palette': palette},
    'NDTI')
Map5.add_colorbar(vis_params, label="Turbidity", layer_name="NDTI")
Map5.addLayerControl()
Map5


EEException: Image.updateMask: If one image has no bands, the other must also have no bands. Got 0 and 1.

###Export Image

In [69]:
visualized_image = NDTI.visualize(min=-0.4, max=0.4, palette=palette)

# Define output path for saving the GeoTIFF in your Google Drive
output_drive_path = '/content/drive/My Drive/NDTI2019.tif'

# Use geemap to export the image directly to your Google Drive
geemap.ee_export_image(visualized_image, filename=output_drive_path, scale=20, region=export_roi, file_per_band=False)


Generating URL ...
Please wait ...
Data downloaded to /content/drive/My Drive/NDTI2019.tif


## Chlorophyll-a Detection Using Normalized Difference Chlorophyll Index (NDCI)

In [74]:
# get NDVI
def getNDCI_coll(image):
  ndci = image.normalizedDifference(['B5', 'B4']).rename('NDCI')
  addNdci = image.addBands(ndci)
  return addNdci

startDate = '2022-01-03'
endDate = '2022-01-07'

S2_images = (
      ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterBounds(roi)
      .filterDate(startDate, endDate)
    )
s2_mosaic = S2_images.mosaic()

NDCI = s2_mosaic.updateMask(ndwiMask).normalizedDifference(['B5', 'B4']).rename('NDCI')


palette = ['red', 'yellow', 'green']
palette = cm.palettes.viridis
vis_params = {'min': -1, 'max': 1, 'palette': palette}

Map5 = geemap.Map()
Map5.center_object(roi)
Map5.addLayer(
    NDCI,
    vis_params,
    #{'palette': palette},
    'NDCI')
Map5.add_colorbar(vis_params, label="Chlorophyll (NDCI)", layer_name="NDCI")
Map5.addLayerControl()
Map5


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

###Export

In [ ]:
visualized_image = NDCI.visualize(min=-1, max=1, palette=palette)

# Define output path for saving the GeoTIFF in your Google Drive
output_drive_path = '/content/drive/My Drive/NDCI2022.tif'

# Use geemap to export the image directly to your Google Drive
geemap.ee_export_image(visualized_image, filename=output_drive_path, scale=20, region=export_roi, file_per_band=False)


Generating URL ...
An error occurred while downloading.
Image.visualize: Expected a string or list of strings for field 'palette'.


## Carbon Monoxide

In [ ]:
S5P_collection = (
      ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CH4')
      .select('CH4_column_volume_mixing_ratio_dry_air_bias_corrected')
      .filterBounds(roi)
      .filterDate(startDate,endDate)
      #.filter(ee.Filter.lt('CLOUD_COVER_LAND', Cloud_limit))
    )

S5P_mosaic = S5P_collection.mosaic()


band_viz = {
  min: 1295,
  max: 2432,
  palette: ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}


In [ ]:
Map7 = geemap.Map()
Map7.center_object(roi)

Map7.addLayer(S5P_mosaic, band_viz)
Map7.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

#Map5.add_colorbar(vis_params, label="Water", layer_name="NDWI")
#Map5.addLayerControl()
Map7

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
S5P_collection = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_CO').select('CO_column_number_density').filterDate(startDate, endDate)

band_viz = {
  min: 0,
  max: 0.05,
  palette: ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

Map8 = geemap.Map()
Map8.center_object(roi)

Map8.addLayer(S5P_collection.mean(), band_viz, 'S5P CO')
Map8

## Air Quality

In [ ]:
AOD_collection = ee.ImageCollection('MODIS/061/MCD19A2_GRANULES').select('Optical_Depth_047').filterDate(startDate, endDate);

band_viz = {
  min: 0,
  max: 1100,
  palette: ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

Map = geemap.Map()
Map.center_object(roi)

Map.addLayer(AOD_collection.mean(), band_viz, 'Optical Depth 047');

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Coral Reef

In [80]:
export_roi = ee.Geometry.Polygon(
            [
                [
            [
              35.36282540361262,
              27.827049821714525
            ],
            [
              35.36282540361262,
              28.26667675726374
            ],
            [
              34.34235686585299,
              28.26667675726374
            ],
            [
              34.34235686585299,
              27.827049821714525
            ],
            [
              35.36282540361262,
              27.827049821714525
            ]
              ]
             ]
            )

In [83]:
startDdae = '2019-01-01'
endDate = '2023-01-01'


dataset = ee.Image('ACA/reef_habitat/v2_0')

Map = geemap.Map()
Map.setOptions('SATELLITE')


reefExtent = dataset.select('reef_mask').selfMask()
Map.addLayer(reefExtent, {}, 'Global reef extent')

geomorphicZonation = dataset.select('geomorphic').selfMask()
Map.addLayer(geomorphicZonation, {}, 'Geomorphic zonation')
Map.center_object(roi)

benthicHabitat = dataset.select('benthic').selfMask()
Map.addLayer(benthicHabitat, {}, 'Benthic habitat')
Map

#coralImage = reefExtent.addBands(geomorphicZonation).addBands(benthicHabitat)


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
visualized_image = NDCI.visualize(min=-1, max=1, palette=palette)

# Define output path for saving the GeoTIFF in your Google Drive
output_drive_path = '/content/drive/My Drive/reefExtent.tif'

# Use geemap to export the image directly to your Google Drive
geemap.ee_export_image(reefExtent, filename=output_drive_path, scale=5, region=export_roi, file_per_band=False)


Generating URL ...
An error occurred while downloading.
Total request size (191003700 bytes) must be less than or equal to 50331648 bytes.


## Land Cover Land Use

In [ ]:
# Construct a collection of corresponding Dynamic World and Sentinel-2 for
# inspection. Filter the DW and S2 collections by region and date.
START = ee.Date('2024-06-01')
END = START.advance(1, 'day')
startDate = '2024-04-09'
endDate = '2024-06-07'

col_filter = ee.Filter.And(
    ee.Filter.bounds(roi),
    ee.Filter.date(startDate, endDate),
)
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(col_filter)
s2_col = ee.ImageCollection('COPERNICUS/S2').filter(col_filter)

# Join corresponding DW and S2 images (by system:index).
dw_s2_col = ee.Join.saveFirst('s2_img').apply(
    dw_col,
    s2_col,
    ee.Filter.equals(leftField='system:index', rightField='system:index'),
)

# Extract an example DW image and its source S2 image.
dw_image = ee.Image(dw_s2_col.first())
s2_image = ee.Image(dw_image.get('s2_img'))
#dw_image = dw_col.mosaic()
#s2_image = s2_col.mosaic()


# Create a visualization that blends DW class label with probability.
# Define list pairs of DW LULC label and color.
CLASS_NAMES = [
    'water',
    'trees',
    'grass',
    'flooded_vegetation',
    'crops',
    'shrub_and_scrub',
    'built',
    'bare',
    'snow_and_ice',
]

VIS_PALETTE = [
    '419bdf',
    '397d49',
    '88b053',
    '7a87c6',
    'e49635',
    'dfc35a',
    'c4281b',
    'a59b8f',
    'b39fe1',
]

# Create an RGB image of the label (most likely class) on [0, 1].
dw_rgb = (
    dw_image.select('label')
    .visualize(min=0, max=8, palette=VIS_PALETTE)
    .divide(255)
)

# Get the most likely class probability.
top1_prob = dw_image.select(CLASS_NAMES).reduce(ee.Reducer.max())

# Create a hillshade of the most likely class probability on [0, 1]
top1_prob_hillshade = ee.Terrain.hillshade(top1_prob.multiply(100)).divide(255)

# Combine the RGB image with the hillshade.
dw_rgb_hillshade = dw_rgb.multiply(top1_prob_hillshade)

# Display the Dynamic World visualization with the source Sentinel-2 image.
m = geemap.Map()
m.set_center(34.34235686585299, 27.827049821714525, 12)
m.add_layer(
    s2_image,
    {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']},
    'Sentinel-2 L1C',
)
m.add_layer(
    dw_rgb_hillshade,
    {'min': 0, 'max': 0.65},
    'Dynamic World V1 - label hillshade',
)
m

Map(center=[27.827049821714525, 34.34235686585299], controls=(WidgetControl(options=['position', 'transparent_…